In [22]:
import os
import pandas as pd
import numpy as np

RES_DIR = "../res"

# Load experiment results

In [23]:
def load_data():
    df = pd.read_parquet(os.path.join(RES_DIR, "all_experiments_results.parquet"))
    npz_data = np.load(os.path.join(RES_DIR, "all_experiments_embeddings.npz"), allow_pickle=True)

    for key in ["distance", "original_image_pred_proba", "embedding_pred_proba", "modified_image_pred_proba", "modified_image"]:
        df[key] = list(npz_data[key])

    df.dropna(axis=1, inplace=True)
    df["input_name"] = df["input_name"].astype(str)
    return df

In [34]:
results = load_data()

In [35]:
results.head(10)

,dataset,delta_multiplier,patch_option,input_name,explored_patches,patch_attribution,number_explored_patches,save_each,algorithm,iteration,...,time,repetition,original_image_pred,embedding_pred,modified_image_pred,distance,original_image_pred_proba,embedding_pred_proba,modified_image_pred_proba,modified_image
0,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,0,...,0.313241,1,0,0,0,[1.7292348994170635e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9889884, -2.4921718, 0.026301473, -2.498427...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
1,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,10,...,0.151353,1,0,0,0,[-1.4062947479575922e-18],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9889836, -2.492174, 0.026299328, -2.4984295...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
2,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,20,...,0.138146,1,0,0,0,[0.0004249268043867243],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9892716, -2.4920435, 0.026858479, -2.497833...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
3,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,30,...,0.147701,1,0,0,0,[1.4900275195070173e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9892716, -2.4920444, 0.02685669, -2.4978378...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
4,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,40,...,0.150380,1,0,0,0,[5.475875917447475e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9892707, -2.4920444, 0.026857853, -2.497834...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
5,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,50,...,0.150639,1,0,0,0,[3.2685708081372906e-20],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.98927, -2.4920425, 0.026857942, -2.4978342,...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
6,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,60,...,0.148689,1,0,0,0,[-3.702896886856977e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9892683, -2.492041, 0.026858687, -2.4978328...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
7,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,70,...,0.148215,1,0,0,0,[-4.345609701723443e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.989267, -2.4920406, 0.026856482, -2.4978318...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
8,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,80,...,0.147460,1,0,0,0,[-1.6439385755233852e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.9892697, -2.4920435, 0.026856542, -2.497835...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"
9,mnist,10.0,one,img_1220,[0],[1.0042133331298828],1,10,simec,90,...,0.147336,1,0,0,0,[8.171448398258515e-19],"[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[6.98927, -2.4920425, 0.026857704, -2.4978344,...","[6.9889874, -2.4921758, 0.02629897, -2.4984267...","[nan, nan, nan, nan]"


In [36]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36360 entries, 0 to 36359
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dataset                    36360 non-null  object 
 1   delta_multiplier           36360 non-null  float64
 2   patch_option               36360 non-null  object 
 3   input_name                 36360 non-null  object 
 4   explored_patches           36360 non-null  object 
 5   patch_attribution          36360 non-null  object 
 6   number_explored_patches    36360 non-null  int64  
 7   save_each                  36360 non-null  int64  
 8   algorithm                  36360 non-null  object 
 9   iteration                  36360 non-null  int64  
 10  delta                      36360 non-null  float64
 11  time                       36360 non-null  float64
 12  repetition                 36360 non-null  int64  
 13  original_image_pred        36360 non-null  int

# Prediction probabilities

Is there a change in top prediction?

In [37]:
results["orig_vs_embed_diff"] = results["original_image_pred"] != results["embedding_pred"]
results["orig_vs_mod_diff"] = results["original_image_pred"] != results["modified_image_pred"]

# Group by the requested columns and aggregate the differences
grouped_res = results.groupby(
    ["dataset", "patch_option", "delta_multiplier", "algorithm"]
).agg(
    orig_vs_embed_diff_count=("orig_vs_embed_diff", "sum"),
    orig_vs_mod_diff_count=("orig_vs_mod_diff", "sum"),
    total=("original_image_pred", "count")
).reset_index()
grouped_res

,dataset,patch_option,delta_multiplier,algorithm,orig_vs_embed_diff_count,orig_vs_mod_diff_count,total
0,mnist,all,0.1,simec,0,0,3030
1,mnist,all,0.1,simexp,0,0,3030
2,mnist,all,10.0,simec,0,0,3030
3,mnist,all,10.0,simexp,59,0,3030
4,mnist,one,0.1,simec,0,0,3030
5,mnist,one,0.1,simexp,0,0,3030
6,mnist,one,10.0,simec,0,0,3030
7,mnist,one,10.0,simexp,0,0,3030
8,mnist,q2,0.1,simec,0,0,3030
9,mnist,q2,0.1,simexp,0,0,3030


Which are the images that change enought to cause a prediction shift? 

In [39]:
# Initialize a list to store the change data
change_records = []

# Define grouping columns
group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

# Group and compare
for group_keys, group in results.groupby(group_cols):
    group = group.sort_values("iteration")
    rows = list(group.itertuples(index=False))
    if not rows:
        continue

    prev_emb = rows[0].embedding_pred
    prev_mod = rows[0].modified_image_pred
    if rows[0].iteration == rows[1].iteration:
        print(rows[0].iteration, group_keys)
        print(rows[1].iteration, group_keys)
    for row in rows[1:]:
        
        changed_emb = row.embedding_pred != prev_emb
        changed_mod = row.modified_image_pred != prev_mod

        if changed_emb or changed_mod:
            change_type = "Both" if changed_emb and changed_mod else ("Emb" if changed_emb else "Mod")

            change_records.append({
                "type": change_type,
                "dataset": row.dataset,
                "algorithm": row.algorithm,
                "delta_multiplier": row.delta_multiplier,
                "patch_option": row.patch_option,
                "input_name": row.input_name,
                "repetition": row.repetition,
                "iteration": row.iteration,
                "original_pred": row.original_image_pred,
                "embedding_pred": row.embedding_pred,
                "modified_pred": row.modified_image_pred
            })

            prev_emb = row.embedding_pred
            prev_mod = row.modified_image_pred

# Convert to DataFrame
df_pred_changes = pd.DataFrame(change_records)

# Show result
df_pred_changes

,type,dataset,algorithm,delta_multiplier,patch_option,input_name,repetition,iteration,original_pred,embedding_pred,modified_pred
0,Emb,mnist,simexp,10.0,all,img_3134,1,999,2,0,2
1,Emb,mnist,simexp,10.0,all,img_3134,3,480,2,0,2
2,Emb,mnist,simexp,10.0,all,img_3737,1,960,8,5,8
3,Emb,mnist,simexp,10.0,q2,img_4341,1,690,2,3,2
4,Emb,mnist,simexp,10.0,q2,img_4341,2,90,2,3,2
5,Emb,mnist,simexp,10.0,q2,img_4341,3,680,2,8,2
6,Emb,mnist,simexp,10.0,q2,img_4341,3,750,2,2,2


At which iteration does the change occur?

In [40]:
# Expanded summary statistics for iteration changes
summary_stats = (
    df_pred_changes
    .groupby(["type", "algorithm", "patch_option", "delta_multiplier"])["iteration"]
    .agg(
        count="count",
        mean="mean",
        std="std",
        min="min",
        max="max",
        median="median",
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75)
    )
    .reset_index()
    .sort_values(["patch_option", "algorithm", "delta_multiplier"])
)

# Display the summary
summary_stats

,type,algorithm,patch_option,delta_multiplier,count,mean,std,min,max,median,q25,q75
0,Emb,simexp,all,10.0,3,813.0,289.044979,480,999,960.0,720.0,979.5
1,Emb,simexp,q2,10.0,4,552.5,309.879009,90,750,685.0,532.5,705.0


Which are the images where the top class has an increasing/decreasing probability?

In [47]:
records = []

group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

for group_keys, group in results.groupby(group_cols):
    group = group.sort_values("iteration")
    rows = list(group.itertuples(index=False))
    if not rows:
        continue

    # Initialize previous preds
    original_pred = rows[0].original_image_pred
    prev_proba_original_pred = rows[0].embedding_pred_proba[original_pred]
    prev_change_type = None

    # Iterate from second row onward
    for row in rows[1:]:
        curr_proba = row.embedding_pred_proba[original_pred]
        if prev_change_type is None:
            if curr_proba < prev_proba_original_pred:
                change_type = "-"
            elif curr_proba > prev_proba_original_pred:
                change_type = "+"
            else:
                change_type = "="
        else:
            if curr_proba < prev_proba_original_pred:
                change_type = "-"
            elif curr_proba > prev_proba_original_pred:
                change_type = "+"
            else:
                change_type = "="

            if change_type != prev_change_type:
                diff = curr_proba - prev_proba_original_pred
                records.append({
                    "type": change_type,
                    "dataset": row.dataset,
                    "algorithm": row.algorithm,
                    "delta_multiplier": row.delta_multiplier,
                    "patch_option": row.patch_option,
                    "input_name": row.input_name,
                    "repetition": row.repetition,
                    "iteration": row.iteration,
                    "diff": diff,
                })

        prev_change_type = change_type

        # Update trackers
        prev_proba_original_pred = curr_proba

# Create DataFrame from collected records
df_changes = pd.DataFrame.from_records(records)
df_changes

,type,dataset,algorithm,delta_multiplier,patch_option,input_name,repetition,iteration,diff
0,+,mnist,simec,0.1,all,img_1220,1,20,4.768372e-07
1,-,mnist,simec,0.1,all,img_1220,1,40,-6.675720e-06
2,+,mnist,simec,0.1,all,img_1220,1,50,2.384186e-06
3,-,mnist,simec,0.1,all,img_1220,1,60,-3.814697e-06
4,+,mnist,simec,0.1,all,img_1220,1,90,9.536743e-07
...,...,...,...,...,...,...,...,...,...
13890,-,mnist,simexp,10.0,q2,img_9185,3,950,-7.144737e-02
13891,+,mnist,simexp,10.0,q2,img_9185,3,960,6.685257e-02
13892,-,mnist,simexp,10.0,q2,img_9185,3,980,-5.742073e-02
13893,+,mnist,simexp,10.0,q2,img_9185,3,990,3.796768e-02


In [48]:
records = []

group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

for group_keys, group in results.groupby(group_cols):
    group = group.sort_values("iteration")
    rows = list(group.itertuples(index=False))
    if not rows:
        continue

    original_pred = rows[0].original_image_pred
    prev_proba = rows[0].embedding_pred_proba[original_pred]

    # Start with no trend
    current_trend = None
    cumulative_diff = 0.0

    for row in rows[1:]:
        curr_proba = row.embedding_pred_proba[original_pred]
        diff = curr_proba - prev_proba

        # Determine direction
        if diff > 0:
            new_trend = "+"
        elif diff < 0:
            new_trend = "-"
        else:
            new_trend = "="

        # Handle direction changes only if meaningful
        if current_trend is None:
            current_trend = new_trend
            cumulative_diff = diff
        elif new_trend == current_trend or new_trend == "=":
            cumulative_diff += diff
        else:
            # Direction changed — check if reversal is stronger than previous trend
            if abs(diff) > abs(cumulative_diff):
                records.append({
                    "type": new_trend,
                    "dataset": row.dataset,
                    "algorithm": row.algorithm,
                    "delta_multiplier": row.delta_multiplier,
                    "patch_option": row.patch_option,
                    "input_name": row.input_name,
                    "repetition": row.repetition,
                    "iteration": row.iteration,
                    "diff": diff,
                })
                current_trend = new_trend
                cumulative_diff = diff
            else:
                cumulative_diff += diff  # Still within old trend

        prev_proba = curr_proba

# Create DataFrame
df_changes = pd.DataFrame.from_records(records)
df_changes

,type,dataset,algorithm,delta_multiplier,patch_option,input_name,repetition,iteration,diff
0,+,mnist,simec,0.1,all,img_1220,2,30,0.000003
1,-,mnist,simec,0.1,all,img_1220,2,180,-0.000004
2,+,mnist,simec,0.1,all,img_1220,2,200,0.000013
3,-,mnist,simec,0.1,all,img_1220,2,220,-0.000001
4,+,mnist,simec,0.1,all,img_1220,2,230,0.000004
...,...,...,...,...,...,...,...,...,...
417,+,mnist,simexp,10.0,q2,img_9185,3,430,0.034691
418,-,mnist,simexp,10.0,q2,img_9185,3,500,-0.126600
419,+,mnist,simexp,10.0,q2,img_9185,3,760,0.064163
420,-,mnist,simexp,10.0,q2,img_9185,3,790,-0.074272


In [57]:
group_cols = ["patch_option", "delta_multiplier", "algorithm", "dataset", "input_name"]
# --- Determine majority change per group ---
majority_summary = (
    df_changes
    .groupby(group_cols + ["type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Determine majority type
majority_summary["majority_type"] = majority_summary[["+", "-"]].idxmax(axis=1)

# Optional: Rename columns for clarity
majority_summary = majority_summary.rename(columns={"+": "count_+", "-": "count_-", "=": "count_="})
pd.set_option('display.max_rows', 500)
# Show the result
majority_summary

type,patch_option,delta_multiplier,algorithm,dataset,input_name,count_+,count_-,majority_type
0,all,0.1,simec,mnist,img_1220,6,6,+
1,all,0.1,simec,mnist,img_2040,0,1,-
2,all,0.1,simec,mnist,img_2211,0,2,-
3,all,0.1,simec,mnist,img_4674,5,3,+
4,all,0.1,simec,mnist,img_8257,6,5,+
5,all,0.1,simec,mnist,img_8354,1,1,+
6,all,0.1,simexp,mnist,img_1647,1,1,+
7,all,0.1,simexp,mnist,img_2211,1,3,-
8,all,0.1,simexp,mnist,img_4674,0,1,-
9,all,0.1,simexp,mnist,img_775,1,1,+


In [52]:
# Count trend types (+, -) per algorithm
trend_counts = (
    df_changes.groupby(["algorithm", "type"])
    .size()
    .unstack(fill_value=0)
    .sort_values(by="+", ascending=False)  # sort by positive trend count
)

display(trend_counts)

type,+,-
algorithm,,
simec,122,123
simexp,80,97


In [53]:
# Average iteration of trend switches (could also look at std deviation)
avg_iter_per_trend = (
    df_changes.groupby(["algorithm", "type"])["iteration"]
    .agg(["mean", "std", "count"])
    .reset_index()
    .sort_values(["algorithm", "type"])
)

display(avg_iter_per_trend)

,algorithm,type,mean,std,count
0,simec,+,281.139344,294.217026,122
1,simec,-,285.121951,302.433751,123
2,simexp,+,246.125000,279.395794,80
3,simexp,-,282.783505,278.656532,97


In [54]:
# Define grouping columns (experiment-level identifiers)
group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

# Assume each experiment has 100 iterations saved (one every 10 iterations for 1000 total iterations)
total_iterations = 101

# Group df_changes by the experiment identifiers plus the change type and count the number of unique iterations
change_summary = (
    df_changes.groupby(group_cols + ["type"])["iteration"]
    .nunique()  # count unique iterations where a change was recorded
    .reset_index(name="change_count")
)

# Pivot the table so that each type ("+", "-", "=") becomes its own column
pivot_summary = change_summary.pivot_table(
    index=group_cols,
    columns="type",
    values="change_count",
    fill_value=0
).reset_index()

# Rename the columns for clarity if present
if "+" in pivot_summary.columns:
    pivot_summary = pivot_summary.rename(columns={"+": "increase_count"})
if "-" in pivot_summary.columns:
    pivot_summary = pivot_summary.rename(columns={"-": "decrease_count"})
if "=" in pivot_summary.columns:
    pivot_summary = pivot_summary.rename(columns={"=": "equal_count"})

# In case a type is missing, add it with 0
for col in ["increase_count", "decrease_count", "equal_count"]:
    if col not in pivot_summary.columns:
        pivot_summary[col] = 0

# Calculate percentage for each change type per experiment
pivot_summary["pct_increase"] = (pivot_summary["increase_count"] / total_iterations) * 100
pivot_summary["pct_decrease"] = (pivot_summary["decrease_count"] / total_iterations) * 100
pivot_summary["pct_equal"] = (pivot_summary["equal_count"] / total_iterations) * 100

# Display the final summary DataFrame
pivot_summary

type,dataset,patch_option,delta_multiplier,algorithm,input_name,repetition,increase_count,decrease_count,equal_count,pct_increase,pct_decrease,pct_equal
0,mnist,all,0.1,simec,img_1220,2,3.0,3.0,0,2.970297,2.970297,0.0
1,mnist,all,0.1,simec,img_1220,3,3.0,3.0,0,2.970297,2.970297,0.0
2,mnist,all,0.1,simec,img_2040,2,0.0,1.0,0,0.000000,0.990099,0.0
3,mnist,all,0.1,simec,img_2211,2,0.0,1.0,0,0.000000,0.990099,0.0
4,mnist,all,0.1,simec,img_2211,3,0.0,1.0,0,0.000000,0.990099,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
160,mnist,q2,10.0,simexp,img_8987,1,2.0,3.0,0,1.980198,2.970297,0.0
161,mnist,q2,10.0,simexp,img_8987,2,3.0,4.0,0,2.970297,3.960396,0.0
162,mnist,q2,10.0,simexp,img_8987,3,0.0,1.0,0,0.000000,0.990099,0.0
163,mnist,q2,10.0,simexp,img_9185,2,1.0,1.0,0,0.990099,0.990099,0.0


In [55]:
high_level_summary = (
    pivot_summary
    .groupby(["algorithm", "patch_option", "delta_multiplier"])
    .agg(
        mean_pct_increase=("pct_increase", "mean"),
        mean_pct_decrease=("pct_decrease", "mean"),
        mean_pct_equal=("pct_equal", "mean"),
        std_pct_increase=("pct_increase", "std"),
        std_pct_decrease=("pct_decrease", "std"),
        std_pct_equal=("pct_equal", "std"),
        num_experiments=("pct_equal", "count")  # or any column, since they're all same length
    )
    .reset_index()
)

# Show the summarized result
high_level_summary

,algorithm,patch_option,delta_multiplier,mean_pct_increase,mean_pct_decrease,mean_pct_equal,std_pct_increase,std_pct_decrease,std_pct_equal,num_experiments
0,simec,all,0.1,1.485149,1.485149,0.0,1.230855,1.076351,0.0,12
1,simec,all,10.0,0.660066,0.770077,0.0,0.700106,0.660066,0.0,9
2,simec,one,0.1,2.292861,2.136529,0.0,1.868483,1.903440,0.0,19
3,simec,one,10.0,0.792079,0.792079,0.0,0.609492,0.688963,0.0,20
4,simec,q2,0.1,1.555870,1.650165,0.0,1.552008,1.606688,0.0,21
5,simec,q2,10.0,0.618812,0.742574,0.0,0.512425,0.458327,0.0,8
6,simexp,all,0.1,0.440044,0.990099,0.0,0.521828,0.495050,0.0,9
7,simexp,all,10.0,0.990099,1.463625,0.0,0.895578,0.587237,0.0,23
8,simexp,one,0.1,1.697313,1.131542,0.0,1.241155,1.331867,0.0,7
9,simexp,one,10.0,0.990099,0.424328,0.0,0.571634,0.529230,0.0,7


In [56]:
# Define the group columns to flatten over iterations and repetitions
group_cols_flat = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name"]

# Count occurrences of each change type
type_counts = (
    df_changes
    .groupby(group_cols_flat + ["type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "- count", "+": "+ count", "=": "= count"})
)

# Compute mean diff for each change type
type_means = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .mean()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "mean -", "+": "mean +", "=": "mean ="})
)

# Compute sumn diff for each change type
type_sums = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .sum()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "sum -", "+": "sum +", "=": "sum ="})
)

# Merge counts and means
summary_df = pd.merge(type_counts, type_means, on=group_cols_flat)

# Merge the result with sums
summary_df = pd.merge(summary_df, type_sums, on=group_cols_flat)
pd.set_option('display.max_rows', 500)
# Display the final summary
summary_df

type,dataset,patch_option,delta_multiplier,algorithm,input_name,+ count,- count,mean +,mean -,sum +,sum -
0,mnist,all,0.1,simec,img_1220,6,6,0.000006,-4.291534e-06,0.000036,-2.574921e-05
1,mnist,all,0.1,simec,img_2040,0,1,0.000000,-5.245209e-06,0.000000,-5.245209e-06
2,mnist,all,0.1,simec,img_2211,0,2,0.000000,-9.536743e-06,0.000000,-1.907349e-05
3,mnist,all,0.1,simec,img_4674,5,3,0.000006,-7.311503e-06,0.000031,-2.193451e-05
4,mnist,all,0.1,simec,img_8257,6,5,0.000006,-7.057190e-06,0.000038,-3.528595e-05
5,mnist,all,0.1,simec,img_8354,1,1,0.000004,-5.722046e-06,0.000004,-5.722046e-06
6,mnist,all,0.1,simexp,img_1647,1,1,0.001893,-3.185272e-04,0.001893,-3.185272e-04
7,mnist,all,0.1,simexp,img_2211,1,3,0.000521,-6.243388e-04,0.000521,-1.873016e-03
8,mnist,all,0.1,simexp,img_4674,0,1,0.000000,-2.288818e-03,0.000000,-2.288818e-03
9,mnist,all,0.1,simexp,img_775,1,1,0.001554,-2.956390e-04,0.001554,-2.956390e-04


In [58]:
# Define the group columns to flatten over iterations and repetitions
group_cols_flat = ["patch_option", "delta_multiplier", "algorithm"]

# Count occurrences of each change type
type_counts = (
    df_changes
    .groupby(group_cols_flat + ["type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "- count", "+": "+ count", "=": "= count"})
)

# Compute mean diff for each change type
type_means = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .mean()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "mean -", "+": "mean +", "=": "mean ="})
)

# Compute sumn diff for each change type
type_sums = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .sum()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "sum -", "+": "sum +", "=": "sum ="})
)

# Merge counts and means
summary_df = pd.merge(type_counts, type_means, on=group_cols_flat)

# Merge the result with sums
summary_df = pd.merge(summary_df, type_sums, on=group_cols_flat)

# Display the final summary
summary_df

summary_df

type,patch_option,delta_multiplier,algorithm,+ count,- count,mean +,mean -,sum +,sum -
0,all,0.1,simec,18,18,0.000006,-0.000006,0.000110,-0.000113
1,all,0.1,simexp,4,9,0.001202,-0.000771,0.004807,-0.006943
2,all,10.0,simec,6,7,0.000956,-0.000704,0.005733,-0.004930
3,all,10.0,simexp,23,34,0.100021,-0.108587,2.300476,-3.691947
4,one,0.1,simec,44,41,0.000004,-0.000004,0.000157,-0.000144
5,one,0.1,simexp,12,8,0.000050,-0.000042,0.000604,-0.000340
6,one,10.0,simec,16,16,0.000191,-0.000091,0.003053,-0.001456
7,one,10.0,simexp,7,3,0.008235,-0.005732,0.057646,-0.017197
8,q2,0.1,simec,33,35,0.000005,-0.000006,0.000169,-0.000208
9,q2,0.1,simexp,8,6,0.000689,-0.000743,0.005512,-0.004460
